In [49]:
import requests
import pandas as pd
import numpy as np
import math
import json
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
from IPython.display import Markdown as md
output_notebook()
url = "http://bing.com/covid/data"
r = requests.get(url)
j = r.json()
#j = r.json()
#print(j)
#df = pd.DataFrame(j['areas'][2])
df = pd.json_normalize(j['areas'][2], 'areas')
#df['totalConfirmed'].sum()
index = df['displayName'].to_list()
totalConfirmed = df['totalConfirmed'].to_list()
totalDeaths = df['totalDeaths'].to_list()
totalRecovered = df['totalRecovered'].to_list()
df2 = pd.DataFrame({'totalConfirmed': totalConfirmed,
                    'totalDeaths': totalDeaths,
                    'totalRecovered': totalRecovered}, index=index)
df2['totalRecovered'].fillna(0, inplace=True)
df2['deathRate'] = df2['totalDeaths'].div(df2['totalConfirmed'], fill_value=0)
df2['recoveryRate'] = df2['totalRecovered'].div(df2['totalConfirmed'], fill_value=0)

Loading BokehJS ...

In [9]:
df.describe()

,totalConfirmed,totalDeaths,totalRecovered,lat,long
count,51.000000,40.000000,16.000000,51.000000,51.000000
mean,1165.470588,19.750000,18.750000,39.577923,-93.475285
std,4296.872860,48.074061,40.824013,6.322399,19.157176
min,39.000000,1.000000,1.000000,19.598518,-155.518555
25%,106.000000,2.000000,1.750000,35.712851,-103.008621
50%,290.000000,6.500000,3.000000,39.356419,-89.656242
75%,753.500000,12.500000,8.500000,43.342552,-78.941738
max,30811.000000,285.000000,124.000000,63.174122,-69.203949


In [10]:
source = ColumnDataSource(data=df2)

total_confirmed = int(df['totalConfirmed'].sum())
total_deaths = int(df['totalDeaths'].sum())
total_recoveries = int(df['totalRecovered'].sum())
death_rate = total_deaths / total_confirmed
recovery_rate = total_recoveries / total_confirmed

title_p = "COVID Confirmed: {:,} - Death Rate: {:.4%} - Recovery Rate: {:.4%}".format(total_confirmed, death_rate, recovery_rate)
title_q = "COVID Deaths: {:,} - Death Rate: {:.4%}".format(total_deaths, death_rate)
title_z = "COVID Recovered: {:,} - Recovery Rate: {:.4%}".format(total_recoveries, recovery_rate)


p = figure(x_range=index, plot_height=800, plot_width=1500, title=title_p)
p.vbar(x='index', top='totalConfirmed', width=0.1, source=source, line_color='white')
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = df['totalConfirmed'].max()+1000
p.xaxis.major_label_orientation = math.pi/4

q = figure(x_range=index, plot_height=800, plot_width=1500, title=title_q)
q.vbar(x='index', top='totalDeaths', width=0.1, source=source, line_color='white')
q.xgrid.grid_line_color = None
q.y_range.start = 0
q.y_range.end = df['totalDeaths'].max()+100
q.xaxis.major_label_orientation = math.pi/4

z = figure(x_range=index, plot_height=800, plot_width=1500, title=title_z)
z.vbar(x='index', top='totalRecovered', width=0.1, source=source, line_color='white')
z.xgrid.grid_line_color = None
z.y_range.start = 0
z.y_range.end = df['totalRecovered'].max()+50
z.xaxis.major_label_orientation = math.pi/4

show(p)
show(q)
show(z)

In [43]:
#ax = df2.plot.bar(rot=45, figsize=(25,5))

df2.head(80)

,totalConfirmed,totalDeaths,totalRecovered,deathRate,recoveryRate
New York,30811,285.0,0.0,0.009250,0.000000
New Jersey,3675,44.0,2.0,0.011973,0.000544
California,2266,42.0,6.0,0.018535,0.002648
Washington,2469,123.0,124.0,0.049818,0.050223
Michigan,1791,24.0,0.0,0.013400,0.000000
Louisiana,1388,46.0,0.0,0.033141,0.000000
Florida,1682,22.0,0.0,0.013080,0.000000
Illinois,1535,16.0,2.0,0.010423,0.001303
Georgia,1026,32.0,0.0,0.031189,0.000000
Massachusetts,1159,11.0,1.0,0.009491,0.000863


In [37]:
df3 = pd.read_json(url)
df3.head(50)

,id,displayName,areas,totalConfirmed,totalDeaths,totalRecovered,lastUpdated
0,world,Global,"{'id': 'chinamainland', 'displayName': 'China ...",453074,20519,113121,2020-03-25T18:52:29.833Z
1,world,Global,"{'id': 'italy', 'displayName': 'Italy', 'areas...",453074,20519,113121,2020-03-25T18:52:29.833Z
2,world,Global,"{'id': 'unitedstates', 'displayName': 'United ...",453074,20519,113121,2020-03-25T18:52:29.833Z
3,world,Global,"{'id': 'spain', 'displayName': 'Spain', 'areas...",453074,20519,113121,2020-03-25T18:52:29.833Z
4,world,Global,"{'id': 'germany', 'displayName': 'Germany', 'a...",453074,20519,113121,2020-03-25T18:52:29.833Z
5,world,Global,"{'id': 'iran', 'displayName': 'Iran', 'areas':...",453074,20519,113121,2020-03-25T18:52:29.833Z
6,world,Global,"{'id': 'france', 'displayName': 'France', 'are...",453074,20519,113121,2020-03-25T18:52:29.833Z
7,world,Global,"{'id': 'switzerland', 'displayName': 'Switzerl...",453074,20519,113121,2020-03-25T18:52:29.833Z
8,world,Global,"{'id': 'southkorea', 'displayName': 'South Kor...",453074,20519,113121,2020-03-25T18:52:29.833Z
9,world,Global,"{'id': 'unitedkingdom', 'displayName': 'United...",453074,20519,113121,2020-03-25T18:52:29.833Z


In [13]:
#pt = df3.pivot_table()

countries = pd.json_normalize(j['areas'])
countries.head()

,id,displayName,areas,totalConfirmed,totalDeaths,totalRecovered,lastUpdated,lat,long,parentId
0,chinamainland,China (mainland),[],81218,3281.0,73650.0,2020-03-25T18:32:26.616Z,36.563114,103.735809,world
1,italy,Italy,"[{'id': 'lombardy_italy', 'displayName': 'Lomb...",74386,7503.0,9362.0,2020-03-25T18:32:26.616Z,43.529028,12.162184,world
2,unitedstates,United States,"[{'id': 'newyork_unitedstates', 'displayName':...",61062,838.0,387.0,2020-03-25T18:32:26.616Z,39.495914,-98.989983,world
3,spain,Spain,"[{'id': 'madrid_spain', 'displayName': 'Madrid...",47610,3434.0,5367.0,2020-03-25T18:32:26.616Z,40.387696,-3.557391,world
4,germany,Germany,"[{'id': 'nrw_germany', 'displayName': 'NRW', '...",35740,186.0,3540.0,2020-03-25T18:32:26.616Z,51.121806,10.400695,world


In [14]:
states = pd.json_normalize(countries['areas'][2])
states.head()

,id,displayName,areas,totalConfirmed,totalDeaths,totalRecovered,lastUpdated,lat,long,parentId
0,newyork_unitedstates,New York,"[{'id': 'newyork_newyork_unitedstates', 'displ...",30811,285.0,NaN,2020-03-25T18:32:26.616Z,42.937080,-75.610695,unitedstates
1,newjersey_unitedstates,New Jersey,"[{'id': 'atlantic_newjersey_unitedstates', 'di...",3675,44.0,2.0,2020-03-25T18:32:26.616Z,40.138779,-74.676910,unitedstates
2,california_unitedstates,California,"[{'id': 'losangeles_california_unitedstates', ...",2266,42.0,6.0,2020-03-25T18:32:26.616Z,37.255100,-119.617523,unitedstates
3,washington_unitedstates,Washington,"[{'id': 'king_washington_unitedstates', 'displ...",2469,123.0,124.0,2020-03-25T18:32:26.616Z,47.411297,-120.556267,unitedstates
4,louisiana_unitedstates,Louisiana,[{'id': 'jeffersonparish_louisiana_unitedstate...,1388,46.0,NaN,2020-03-25T18:32:26.616Z,30.966511,-91.851906,unitedstates


In [52]:
counties = pd.json_normalize(data=j['areas'][2]['areas'], record_path='areas')
counties.head(100)

,id,displayName,areas,totalConfirmed,totalDeaths,totalRecovered,lastUpdated,lat,long,parentId
0,newyork_newyork_unitedstates,New York,[],17856,NaN,NaN,2020-03-25T18:57:37.994Z,40.774364,-73.969772,newyork_unitedstates
1,westchester_newyork_unitedstates,Westchester,[],4691,NaN,NaN,2020-03-25T18:57:37.994Z,41.151657,-73.753326,newyork_unitedstates
2,nassau_newyork_unitedstates,Nassau,[],3285,NaN,NaN,2020-03-25T18:57:37.994Z,40.756714,-73.591393,newyork_unitedstates
3,suffolk_newyork_unitedstates,Suffolk,[],2260,NaN,NaN,2020-03-25T18:57:37.994Z,40.966293,-72.687111,newyork_unitedstates
4,rockland_newyork_unitedstates,Rockland,[],968,NaN,NaN,2020-03-25T18:57:37.994Z,41.152374,-74.024010,newyork_unitedstates
...,...,...,...,...,...,...,...,...,...,...
95,fresno_california_unitedstates,Fresno,[],13,NaN,NaN,2020-03-25T18:57:37.994Z,36.758274,-119.649193,california_unitedstates
96,stanislaus_california_unitedstates,Stanislaus,[],11,NaN,NaN,2020-03-25T18:57:37.994Z,37.559326,-120.997665,california_unitedstates
97,imperial_california_unitedstates,Imperial,[],6,NaN,NaN,2020-03-25T18:57:37.994Z,33.039684,-115.365303,california_unitedstates
98,madera_california_unitedstates,Madera,[],6,NaN,1.0,2020-03-25T18:57:37.994Z,37.218010,-119.762726,california_unitedstates


In [51]:
tennessee = "The current death rate in Tennessee is **`{:.4%}`**".format(df2.loc['Tennessee', 'deathRate'])
md(tennessee)

The current death rate in Tennessee is **`0.2999%`**